# Plate Mapper
Reads input excel file and calculates number of reactions

In [ ]:
# Import the necessary libraries, including os and pandas
import os
import pandas as pd
import math

def mapped_reactions(reactions):
    
    if reactions % 2 == 0:
        return 0
    else:
        return 1


def uL_for_10ug(ng_per_uL):
    ul_limit = round(min(10000/ng_per_uL, 25.0), 1)
    return ul_limit

# Load the Excel file 'input.xlsx' into a DataFrame
input_df = pd.read_excel('input.xlsx', engine='openpyxl')

input_df.index =input_df.index + 1

#Calculate amount of DNA in sample
input_df['DNA ug'] = input_df['Volume'] * (input_df['Concentration nanograms'] / 1000)

#Calculate coverage
input_df['ug of DNA needed'] = (input_df['number of guides'].astype(float) * 0.006 * input_df['Coverage']) / 1000


#TODO
#Will need to limit each reaction to 10ug/rxn then find minimum number of reactions to reach desired coverage
input_df['10ug_limit'] = input_df['Concentration nanograms'].apply(uL_for_10ug)

#round(min(10000/ng_per_uL, 25.0), 1)


#Round up to nearest whole number
input_df['number of reactions'] = (((input_df['Volume'] * input_df['Concentration nanograms']) / 1000 ) / input_df['ug of DNA needed']).apply(lambda x: math.ceil(x))

#Add addtional reaction to odd numbers so that plate maps are evenly spaced
input_df['empty reactions'] = input_df['number of reactions'].apply(mapped_reactions)

print(input_df)



#has the extra reaction
rxn_df = input_df[['Sample','number of reactions', 'empty reactions']].copy()


rxn_list = rxn_df.values.tolist()




ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Create master list of samples and reactions to be formatted into map.  Create 'empty' spacers during this step

In [133]:
print(rxn_list)

sample_rxn_list = []
for sample in rxn_list:
    for rxn in range(sample[1]):
        sample_rxn_list.append(sample[0])
    if sample[2] == 1:
        sample_rxn_list.append('empty')

print(sample_rxn_list)

[['control', 5, 1], ['test1', 5, 1], ['test2', 6, 0], ['test3', 4, 0], ['test4', 4, 0]]
['control', 'control', 'control', 'control', 'control', 'empty', 'test1', 'test1', 'test1', 'test1', 'test1', 'empty', 'test2', 'test2', 'test2', 'test2', 'test2', 'test2', 'test3', 'test3', 'test3', 'test3', 'test4', 'test4', 'test4', 'test4']


Map wells to sample names

In [134]:
well_cols = list(range(1,13))
well_rows = ['A','B','C','D','E','F','G','H']

well_dict = {}

for row in well_rows:
    for col in well_cols:
        well_dict[row + str(col)] ='empty'


for i, well in enumerate(well_dict):
    try:
        well_dict[well] = sample_rxn_list[i]
    except:
        well_dict[well] = 'empty'

print(well_dict)



{'A1': 'control', 'A2': 'control', 'A3': 'control', 'A4': 'control', 'A5': 'control', 'A6': 'empty', 'A7': 'test1', 'A8': 'test1', 'A9': 'test1', 'A10': 'test1', 'A11': 'test1', 'A12': 'empty', 'B1': 'test2', 'B2': 'test2', 'B3': 'test2', 'B4': 'test2', 'B5': 'test2', 'B6': 'test2', 'B7': 'test3', 'B8': 'test3', 'B9': 'test3', 'B10': 'test3', 'B11': 'test4', 'B12': 'test4', 'C1': 'test4', 'C2': 'test4', 'C3': 'empty', 'C4': 'empty', 'C5': 'empty', 'C6': 'empty', 'C7': 'empty', 'C8': 'empty', 'C9': 'empty', 'C10': 'empty', 'C11': 'empty', 'C12': 'empty', 'D1': 'empty', 'D2': 'empty', 'D3': 'empty', 'D4': 'empty', 'D5': 'empty', 'D6': 'empty', 'D7': 'empty', 'D8': 'empty', 'D9': 'empty', 'D10': 'empty', 'D11': 'empty', 'D12': 'empty', 'E1': 'empty', 'E2': 'empty', 'E3': 'empty', 'E4': 'empty', 'E5': 'empty', 'E6': 'empty', 'E7': 'empty', 'E8': 'empty', 'E9': 'empty', 'E10': 'empty', 'E11': 'empty', 'E12': 'empty', 'F1': 'empty', 'F2': 'empty', 'F3': 'empty', 'F4': 'empty', 'F5': 'empty',

Create a dict of row specific lists then split each sample based on well_dict.key into specific row list

In [136]:
rows_dict = {}
#generating a dict of rows ord() returns the unicode value of a character, chr() returns the character of a unicode value
for i in range(ord('A'), ord('I')):
    row = chr(i)
    rows_dict[row] = []
    
#filling the rows with the well_dict values
for well in well_dict.items():
    for row in rows_dict.items():
        if well[0][0] == row[0]:
            row[1].append(well[1])
            
#converting the dict to a dataframe
plate_layout = pd.DataFrame(rows_dict)
#transposing the dataframe, going from long to wide
plate_layout = plate_layout.T

#renumber columns to match plate
plate_layout.columns = list(range(1,13))

print(plate_layout)

plate_layout.to_excel('plate_layout.xlsx', engine='openpyxl')

        1        2        3        4        5      6      7      8      9   \
A  control  control  control  control  control  empty  test1  test1  test1   
B    test2    test2    test2    test2    test2  test2  test3  test3  test3   
C    test4    test4    empty    empty    empty  empty  empty  empty  empty   
D    empty    empty    empty    empty    empty  empty  empty  empty  empty   
E    empty    empty    empty    empty    empty  empty  empty  empty  empty   
F    empty    empty    empty    empty    empty  empty  empty  empty  empty   
G    empty    empty    empty    empty    empty  empty  empty  empty  empty   
H    empty    empty    empty    empty    empty  empty  empty  empty  empty   

      10     11     12  
A  test1  test1  empty  
B  test3  test4  test4  
C  empty  empty  empty  
D  empty  empty  empty  
E  empty  empty  empty  
F  empty  empty  empty  
G  empty  empty  empty  
H  empty  empty  empty  
